In [1]:
# Task 2 is simply to make a list of all of the variables in the JAN2021 ICEES+ COVID instance at https://covid.icees.renci.org/jan2021/query
# that are either missing (e.g.,in all_features.yaml and the corresponding Dhall files,
# but not exposed via the API) or showing empty cells (e.g., counts of all zeroes).

import yaml
import requests
import json

#fdir: output file directory
#fname: output file name

def Variables_missingvalues(fdir,fname):
    resp = requests.get("https://covid.icees.renci.org/jan2021/patient/cohort/COHORT:1/features")
    obj = resp.json()

    features = obj["return value"]

    null_only_features = []

    for feature_data in features:
        feature_qualifiers = feature_data["feature"]["feature_qualifiers"]
        feature_matrix = feature_data["feature_matrix"]
        null_indices = []

        for i in range(len(feature_qualifiers)):
            operator = feature_qualifiers[i]["operator"]
            value = feature_qualifiers[i]["value"]
            if operator == "=" and value in [0, None]:
                null_indices.append(i)

        has_only_null_values = True
        for index in range(len(feature_matrix)):
            if index not in null_indices and feature_matrix[index]["frequency"] > 0:
                has_only_null_values = False

        if has_only_null_values:
            null_only_features.append(feature_data["feature"]["feature_name"])

    featuresNAList_file = open(fdir + "\\" + fname + ".yaml", 'w')
    yaml.dump(null_only_features, featuresNAList_file)
    featuresNAList_file.close()  
    #print(null_only_features)
    #print("\n".join(null_only_features))
    
Variables_missingvalues("D:\BL\Renci\icees-api-config\Python-BL", "Variables_with_MissingValue")
    